In [ ]:
!pip install lightgbm
from lightgbm import LGBMClassifier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.8 MB/s eta 0:00:00


In [101]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    path_to_file = '/content/gdrive/My Drive/BT5153_project'
    print(path_to_file)
    # move to Google Drive directory
    os.chdir(path_to_file)
    !pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/BT5153_project
/content/gdrive/My Drive/BT5153_project


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [65]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
# from sentence_transformers import SentenceTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import cross_validate
!pip install --quiet transformers
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from transformers import AutoTokenizer
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')

In [ ]:
from datetime import datetime
import re

# Define the function to parse dates from experience text
def parse_dates(text):
    # Regular expression to find date ranges like '2020.7-2022.12' or '2022.12-.'
    date_ranges = re.findall(r'(\d{4}\.\d{1,2})-(\d{4}\.\d{1,2}|.)?', text)

    date_pairs = []
    for start, end in date_ranges:
        start_date = datetime.strptime(start, "%Y.%m")
        if end == '.' or end == '':  # Ongoing job, no end date specified
            end_date = datetime.now()  # Use current time as end date
        else:
            try:
                end_date = datetime.strptime(end, "%Y.%m")
            except ValueError:
                continue
        date_pairs.append((start_date, end_date))
    return date_pairs

# Define the function to calculate duration in months from date pairs
def calculate_durations(date_pairs):
    durations = []
    for start, end in date_pairs:
        month_difference = (end.year - start.year) * 12 + (end.month - start.month)
        durations.append(month_difference)
    return sum(durations)


# Define a function to calculate total work duration from experience text
def total_experience(text):
    try:
        if isinstance(text, str):
            date_pairs = parse_dates(text)
            total_months = calculate_durations(date_pairs)
            return total_months / 12  # Convert months to years
        return np.nan  # Return NaN for non-string inputs
    except:
        print(text)

# Apply the function to calculate total work experience for each user


In [ ]:
mix = pd.read_csv('mix_combined.csv',index_col=0)

In [ ]:
# mix['total_experience_years'] = mix['experiences'].apply(total_experience)
# mix[['total_experience_years','experiences']].iloc[2]

total_experience_years                                                57.75
experiences               JLab Audio : President\n2011.11-.\nNo descript...
Name: 1115736, dtype: object

In [ ]:
# mix[['total_experience_years','experiences']].iloc[2]

total_experience_years                                                57.75
experiences               JLab Audio : President\n2011.11-.\nNo descript...
Name: 1115736, dtype: object

In [ ]:
# print(mix[['total_experience_years','experiences']].iloc[2]['experiences'])

In [ ]:
encoded_input = tokenizer(list(mix['combined_text']),padding = True)

In [ ]:
input_embed = pd.DataFrame(encoded_input['input_ids'])
input_embed.index = mix.index
input_embed

,0,1,2,3,4,5,6,7,8,9,...,2815,2816,2817,2818,2819,2820,2821,2822,2823,2824
user_id,,,,,,,,,,,,,,,,,,,,,
1033137,101,12654,1024,2472,1997,16012,21890,17830,10067,2470,...,0,0,0,0,0,0,0,0,0,0
1098586,101,12654,1024,8744,1025,2495,1024,27302,12900,6894,...,0,0,0,0,0,0,0,0,0,0
1115736,101,12654,1024,1045,2572,1996,5766,1997,1046,20470,...,0,0,0,0,0,0,0,0,0,0
1341457,101,12654,1024,1045,3749,1066,2322,2086,1521,3325,...,0,0,0,0,0,0,0,0,0,0
1501168,101,12654,1024,2057,2569,4697,1999,4346,2146,1011,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780766472,101,12654,1024,1528,3026,2658,2007,2570,2086,1997,...,0,0,0,0,0,0,0,0,0,0
794835922,101,12654,1024,3716,3085,7177,24555,2007,2184,1009,...,0,0,0,0,0,0,0,0,0,0
794846268,101,12654,1024,8744,1025,2495,1024,27935,8665,2082,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
y_col = mix.columns[7:7+50]
lang_col =  mix.columns[-59-1:-1]
feat_col = ['num of languages','num of education','num of experiences']

In [ ]:
X = pd.concat([mix[lang_col],mix[feat_col],input_embed], axis=1)
y = mix[y_col]
X.columns = X.columns.astype(str)

In [ ]:
preprocessor = ColumnTransformer(
    # (nickname, transformer to apply, columns to apply to)
    transformers=[

        ('feat', StandardScaler(), feat_col),
        ('lang',  PCA(n_components=0.5), lang_col),
        ('embedding',  PCA(n_components=0.75), X.columns[-2825 :])
    ],
)


forest = RandomForestClassifier()
multi_target_forest = MultiOutputClassifier(forest, n_jobs=None)



In [ ]:
preprocessor

ColumnTransformer(transformers=[('feat', StandardScaler(),
                                 ['num of languages', 'num of education',
                                  'num of experiences']),
                                ('lang', PCA(n_components=0.5),
                                 Index(['language_afrikaans', 'language_american',
       'language_american sign language', 'language_amharic',
       'language_ancient greek', 'language_arabic', 'language_bahasa malaysia',
       'language_bengali', 'language_bloomberg',...
       'language_tagalog', 'language_taiwanese', 'language_tamil',
       'language_telugu', 'language_thai', 'language_tigrinya',
       'language_turkish', 'language_ukrainian', 'language_urdu',
       'language_vietnamese'],
      dtype='object')),
                                ('embedding', PCA(n_components=0.75),
                                 Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '2815', '2816', '2817', '2818', '2819', '2820', '2821', '2822', '2823',
       '2824'],
      dtype='object', length=2825))])

In [ ]:
X_processed = pd.DataFrame(
    data = preprocessor.fit_transform(X),
    columns = preprocessor.get_feature_names_out(), # <- get the encoded feature names
    index = X.index
)

In [ ]:
X_processed.columns

Index(['feat__num of languages', 'feat__num of education',
       'feat__num of experiences', 'lang__pca0', 'lang__pca1',
       'embedding__pca0', 'embedding__pca1', 'embedding__pca2',
       'embedding__pca3', 'embedding__pca4',
       ...
       'embedding__pca300', 'embedding__pca301', 'embedding__pca302',
       'embedding__pca303', 'embedding__pca304', 'embedding__pca305',
       'embedding__pca306', 'embedding__pca307', 'embedding__pca308',
       'embedding__pca309'],
      dtype='object', length=315)

In [ ]:
f1_macro = make_scorer(f1_score,average='macro')
f1_micro = make_scorer(f1_score,average='micro')
f1_weighted = make_scorer(f1_score,average='weighted')
f1_samples = make_scorer(f1_score,average='samples')


In [ ]:


forest = RandomForestClassifier()
multi_target_forest = MultiOutputClassifier(forest, n_jobs=None)

# Perform cross-validation
scoring = {'accuracy': 'accuracy','f1_macro':f1_macro,'f1_micro':f1_micro,'f1_weighted':f1_weighted,'f1_samples':f1_samples}
cv_results = cross_validate(multi_target_forest, X_processed, y, cv=4, scoring=scoring,return_train_score = True)

cv_results = pd.DataFrame(cv_results)
cv_results_mean = cv_results.mean()
cv_results.loc['mean'] = cv_results_mean
print(cv_results)

In [ ]:
cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro,test_f1_weighted,train_f1_weighted,test_f1_samples,train_f1_samples
0,217.612362,0.858988,0.149941,0.998032,0.042518,0.999519,0.427493,0.999765,0.303266,0.999764,0.520784,0.999744
1,223.641028,0.858936,0.172373,0.998032,0.039756,0.999190,0.427116,0.999768,0.303279,0.999766,0.522307,0.999715
2,222.577090,0.846173,0.184179,0.998819,0.040408,0.979628,0.426491,0.999858,0.304960,0.999858,0.527436,0.999892
3,218.132202,0.872332,0.152302,0.998426,0.048486,0.999728,0.428014,0.999811,0.306907,0.999810,0.521483,0.999746
mean,220.490670,0.859107,0.164699,0.998327,0.042792,0.994516,0.427279,0.999801,0.304603,0.999799,0.523003,0.999774


In [ ]:
forest = RandomForestClassifier(n_estimators = 300,min_samples_split=10)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=None)

scoring = {'accuracy': 'accuracy','f1_macro':f1_macro,'f1_micro':f1_micro,'f1_weighted':f1_weighted,'f1_samples':f1_samples}
cv_results = cross_validate(multi_target_forest, X_processed, y, cv=5, scoring=scoring,return_train_score = True)

cv_results = pd.DataFrame(cv_results)
cv_results_mean = cv_results.mean()
cv_results.loc['mean'] = cv_results_mean
print(cv_results)

In [ ]:
cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro,test_f1_weighted,train_f1_weighted
0,611.200414,2.475828,0.159292,0.484871,0.043589,0.392747,0.431490,0.819466,0.307647,0.761440
1,634.006892,2.127126,0.179941,0.478229,0.039464,0.402087,0.433868,0.820515,0.304066,0.764122
2,589.631039,2.138927,0.172566,0.475277,0.042462,0.407300,0.435623,0.821009,0.313628,0.765086
3,580.273151,2.195455,0.166913,0.482110,0.039536,0.402671,0.429693,0.823505,0.305454,0.766667
4,574.181320,2.112282,0.156573,0.485799,0.054698,0.397281,0.423114,0.821605,0.305055,0.764940
mean,597.858563,2.209924,0.167057,0.481257,0.043950,0.400417,0.430758,0.821220,0.307170,0.764451


In [ ]:
LGBM = LGBMClassifier()
multi_target_LGBM = MultiOutputClassifier(LGBM, n_jobs=None)

scoring = {'accuracy': 'accuracy','f1_macro':f1_macro,'f1_micro':f1_micro,'f1_weighted':f1_weighted,'f1_samples':f1_samples}
cv_results = cross_validate(multi_target_LGBM, X_processed, y, cv=5, scoring=scoring,return_train_score = True)

cv_results = pd.DataFrame(cv_results)
cv_results_mean = cv_results.mean()
cv_results.loc['mean'] = cv_results_mean
print(cv_results)

In [ ]:
cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro,test_f1_weighted,train_f1_weighted,test_f1_samples,train_f1_samples
0,28.581681,0.177545,0.146018,1.000000,0.041613,1.000000,0.427204,1.000000,0.302707,1.000000,0.519346,1.000000
1,28.529540,0.180226,0.154867,0.999631,0.040122,0.999991,0.427286,0.999956,0.301268,0.999956,0.518540,0.999959
2,29.042503,0.178034,0.179941,1.000000,0.038596,1.000000,0.434001,1.000000,0.308362,1.000000,0.531181,1.000000
3,28.225401,0.176728,0.155096,0.999262,0.039483,0.959992,0.429006,0.999912,0.304973,0.999868,0.527935,0.999925
4,28.684761,0.175252,0.153619,1.000000,0.038549,1.000000,0.416667,1.000000,0.290524,1.000000,0.514893,1.000000
mean,28.612777,0.177557,0.157908,0.999779,0.039672,0.991997,0.426833,0.999974,0.301567,0.999965,0.522379,0.999977


In [ ]:
preprocessor = ColumnTransformer(
    # (nickname, transformer to apply, columns to apply to)
    transformers=[

        ('feat', StandardScaler(), feat_col),
        ('lang',  PCA(n_components=0.5), lang_col),
        ('embedding',  PCA(n_components=0.85), X.columns[-2825 :])
    ],
)

X_processed = pd.DataFrame(
    data = preprocessor.fit_transform(X),
    columns = preprocessor.get_feature_names_out(), # <- get the encoded feature names
    index = X.index
)
X_processed.columns

Index(['feat__num of languages', 'feat__num of education',
       'feat__num of experiences', 'lang__pca0', 'lang__pca1',
       'embedding__pca0', 'embedding__pca1', 'embedding__pca2',
       'embedding__pca3', 'embedding__pca4',
       ...
       'embedding__pca420', 'embedding__pca421', 'embedding__pca422',
       'embedding__pca423', 'embedding__pca424', 'embedding__pca425',
       'embedding__pca426', 'embedding__pca427', 'embedding__pca428',
       'embedding__pca429'],
      dtype='object', length=435)

In [ ]:
LGBM = LGBMClassifier()
multi_target_LGBM = MultiOutputClassifier(LGBM, n_jobs=None)

scoring = {'accuracy': 'accuracy','f1_macro':f1_macro,'f1_micro':f1_micro,'f1_weighted':f1_weighted,'f1_samples':f1_samples}
cv_results = cross_validate(multi_target_LGBM, X_processed, y, cv=5, scoring=scoring,return_train_score = True)

cv_results = pd.DataFrame(cv_results)
cv_results_mean = cv_results.mean()
cv_results.loc['mean'] = cv_results_mean
cv_results

In [ ]:
cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro,test_f1_weighted,train_f1_weighted,test_f1_samples,train_f1_samples
0,32.726546,0.228608,0.147493,1.000000,0.040337,1.000,0.430005,1.000000,0.303351,1.000000,0.520632,1.000000
1,32.462873,0.196097,0.162242,1.000000,0.041428,1.000,0.436764,1.000000,0.307902,1.000000,0.529560,1.000000
2,33.038496,0.199236,0.175516,1.000000,0.039332,1.000,0.436036,1.000000,0.309469,1.000000,0.532812,1.000000
3,32.639534,0.193178,0.162482,0.999631,0.039145,0.960,0.432942,0.999956,0.306475,0.999912,0.532087,0.999966
4,32.377914,0.227996,0.138848,1.000000,0.037896,1.000,0.413861,1.000000,0.287729,1.000000,0.510787,1.000000
mean,32.649073,0.209023,0.157316,0.999926,0.039628,0.992,0.429922,0.999991,0.302985,0.999982,0.525176,0.999993


In [ ]:
preprocessor = ColumnTransformer(
    # (nickname, transformer to apply, columns to apply to)
    transformers=[

        ('feat', StandardScaler(), feat_col),
        ('lang',  PCA(n_components=0.5), lang_col),
        ('embedding',  PCA(n_components=0.95), X.columns[-2825 :])
    ],
)

X_processed = pd.DataFrame(
    data = preprocessor.fit_transform(X),
    columns = preprocessor.get_feature_names_out(), # <- get the encoded feature names
    index = X.index
)
X_processed.columns

Index(['feat__num of languages', 'feat__num of education',
       'feat__num of experiences', 'lang__pca0', 'lang__pca1',
       'embedding__pca0', 'embedding__pca1', 'embedding__pca2',
       'embedding__pca3', 'embedding__pca4',
       ...
       'embedding__pca628', 'embedding__pca629', 'embedding__pca630',
       'embedding__pca631', 'embedding__pca632', 'embedding__pca633',
       'embedding__pca634', 'embedding__pca635', 'embedding__pca636',
       'embedding__pca637'],
      dtype='object', length=643)

In [ ]:
LGBM = LGBMClassifier()
multi_target_LGBM = MultiOutputClassifier(LGBM, n_jobs=None)

scoring = {'accuracy': 'accuracy','f1_macro':f1_macro,'f1_micro':f1_micro,'f1_weighted':f1_weighted,'f1_samples':f1_samples}
cv_results = cross_validate(multi_target_LGBM, X_processed, y, cv=5, scoring=scoring,return_train_score = True)

cv_results = pd.DataFrame(cv_results)
cv_results_mean = cv_results.mean()
cv_results.loc['mean'] = cv_results_mean


In [ ]:
cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro,test_f1_weighted,train_f1_weighted,test_f1_samples,train_f1_samples
0,40.803916,0.232183,0.163717,1.000000,0.039025,1.000,0.426989,1.000000,0.299703,1.000000,0.522159,1.000000
1,40.933221,0.236935,0.153392,1.000000,0.038271,1.000,0.425308,1.000000,0.297093,1.000000,0.520435,1.000000
2,40.566969,0.238702,0.174041,1.000000,0.038914,1.000,0.435142,1.000000,0.308440,1.000000,0.530991,1.000000
3,39.501939,0.229960,0.153619,0.925858,0.039258,0.960,0.423482,0.991247,0.304452,0.999912,0.520838,0.987142
4,40.166187,0.231302,0.144756,1.000000,0.037553,1.000,0.413438,1.000000,0.287354,1.000000,0.511763,1.000000
mean,40.394446,0.233817,0.157905,0.985172,0.038604,0.992,0.424872,0.998249,0.299408,0.999982,0.521237,0.997428


In [ ]:
preprocessor = ColumnTransformer(
    # (nickname, transformer to apply, columns to apply to)
    transformers=[

        ('feat', StandardScaler(), feat_col),
        ('lang',  PCA(n_components=0.5), lang_col),
        ('embedding',  PCA(n_components=0.5), X.columns[-2825 :])
    ],
)

X_processed = pd.DataFrame(
    data = preprocessor.fit_transform(X),
    columns = preprocessor.get_feature_names_out(), # <- get the encoded feature names
    index = X.index
)
X_processed.columns

Index(['feat__num of languages', 'feat__num of education',
       'feat__num of experiences', 'lang__pca0', 'lang__pca1',
       'embedding__pca0', 'embedding__pca1', 'embedding__pca2',
       'embedding__pca3', 'embedding__pca4',
       ...
       'embedding__pca113', 'embedding__pca114', 'embedding__pca115',
       'embedding__pca116', 'embedding__pca117', 'embedding__pca118',
       'embedding__pca119', 'embedding__pca120', 'embedding__pca121',
       'embedding__pca122'],
      dtype='object', length=128)

In [ ]:
LGBM = LGBMClassifier()
multi_target_LGBM = MultiOutputClassifier(LGBM, n_jobs=None)

scoring = {'accuracy': 'accuracy','f1_macro':f1_macro,'f1_micro':f1_micro,'f1_weighted':f1_weighted,'f1_samples':f1_samples}
cv_results = cross_validate(multi_target_LGBM, X_processed, y, cv=5, scoring=scoring,return_train_score = True)

cv_results = pd.DataFrame(cv_results)
cv_results_mean = cv_results.mean()
cv_results.loc['mean'] = cv_results_mean


In [ ]:
cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro,test_f1_weighted,train_f1_weighted,test_f1_samples,train_f1_samples
0,22.866700,0.181048,0.141593,0.998524,0.039469,0.999966,0.417776,0.999824,0.296089,0.999824,0.509234,0.999709
1,22.139735,0.145262,0.148968,0.996679,0.041287,0.993255,0.426805,0.999559,0.304050,0.999544,0.515816,0.999562
2,21.885941,0.147107,0.179941,0.996310,0.041212,0.987940,0.440513,0.999563,0.316770,0.999589,0.535994,0.999468
3,21.007163,0.141698,0.158050,0.997787,0.040994,0.979948,0.429902,0.999736,0.307863,0.999736,0.526486,0.999691
4,21.289208,0.146973,0.140325,0.999262,0.038782,0.999983,0.414622,0.999911,0.290149,0.999911,0.508517,0.999923
mean,21.837749,0.152418,0.153775,0.997712,0.040349,0.992218,0.425924,0.999719,0.302984,0.999721,0.519210,0.999671


In [ ]:
scoring

{'accuracy': 'accuracy',
 'f1_macro': make_scorer(f1_score, average=macro),
 'f1_micro': make_scorer(f1_score, average=micro),
 'f1_weighted': make_scorer(f1_score, average=weighted),
 'f1_samples': make_scorer(f1_score, average=samples)}

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'estimator__n_estimators': [100, 200, 300],
    'estimator__learning_rate': [0.01, 0.1, 0.2],
    'estimator__max_depth': [3, 5, 7]
}

LGBM = LGBMClassifier()
multi_target_LGBM = MultiOutputClassifier(LGBM, n_jobs=None)

# Create a GridSearchCV instance
grid_search = GridSearchCV(multi_target_LGBM, param_grid, cv=5, scoring=scoring, refit = 'f1_samples',return_train_score=True)

# Fit the GridSearchCV instance to the data
grid_search.fit(X_processed, y)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best parameters:", best_params)
print("Best score:", best_score)

In [97]:
print("Best parameters:", best_params)
print("Best score:", best_score)

Best parameters: {'estimator__learning_rate': 0.01, 'estimator__max_depth': 3, 'estimator__n_estimators': 300}
Best score: 0.5258478881767463


In [170]:
LGBM = LGBMClassifier(learning_rate=0.1,max_depth=3,n_estimators=300,verbose = -1)
multi_target_LGBM = MultiOutputClassifier(LGBM, n_jobs=None)
multi_target_LGBM

MultiOutputClassifier(estimator=LGBMClassifier(max_depth=3, n_estimators=300,
                                               verbose=-1))

In [167]:
scoring = {'accuracy': 'accuracy','f1_macro':f1_macro,'f1_micro':f1_micro,'f1_weighted':f1_weighted,'f1_samples':f1_samples}
cv_results = cross_validate(multi_target_LGBM, X_processed, y, cv=5, scoring=scoring,return_train_score = True)

cv_results = pd.DataFrame(cv_results)
cv_results_mean = cv_results.mean()
cv_results.loc['mean'] = cv_results_mean


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-

In [168]:
cv_results

,fit_time,score_time,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro,test_f1_weighted,train_f1_weighted,test_f1_samples,train_f1_samples
0,16.629094,0.159375,0.141593,0.833948,0.045444,0.987468,0.428077,0.968495,0.310939,0.967192,0.515084,0.960872
1,16.700859,0.156794,0.148968,0.818081,0.045962,0.965183,0.433687,0.963743,0.312843,0.961990,0.520772,0.954607
2,16.776421,0.156432,0.168142,0.821033,0.046505,0.985183,0.441466,0.964208,0.324526,0.962521,0.533138,0.953855
3,16.667612,0.170408,0.163959,0.814460,0.047209,0.965622,0.434783,0.964647,0.319728,0.962985,0.529398,0.954330
4,16.204763,0.151412,0.129985,0.825895,0.046687,0.985771,0.423761,0.965445,0.305610,0.963943,0.511540,0.957473
mean,16.595750,0.158884,0.150529,0.822683,0.046361,0.977846,0.432355,0.965308,0.314729,0.963726,0.521986,0.956227


In [171]:
X_train, X_test, y_train, y_test = train_test_split(X_processed,y, test_size=0.2, random_state=42)

multi_target_LGBM.fit(X_train,y_train)

y_pred=multi_target_LGBM.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test,y_pred)
print(f"Accuracy:{accuracy:.4f}")
# Calculate precision,recall,and F1-scoreprint(classification report(y test, y pred, target names=y test.columns))

Accuracy:0.1372


In [172]:
f1_macro(multi_target_LGBM,X_test,y_test)

0.049105538965915685

In [173]:
f1_weighted(multi_target_LGBM,X_test,y_test)

0.3075671355103483

In [174]:
f1_micro(multi_target_LGBM,X_test,y_test)

0.4251439539347409

In [175]:
f1_samples(multi_target_LGBM,X_test,y_test)

0.5132449230835487

In [176]:
from sklearn.metrics import classification_report

In [177]:
print(classification_report(y_test, y_pred,target_names=y_col))

                                                                                       precision    recall  f1-score   support

                                skill_k50_accounting / financial reporting / auditing       0.20      0.01      0.02        81
                                    skill_k50_analysis / financial analysis / finance       0.91      1.00      0.95       613
                              skill_k50_autocad / solidworks / mechanical engineering       0.00      0.00      0.00         8
                                               skill_k50_banking / insurance / credit       0.17      0.01      0.02       115
                                  skill_k50_biotechnology / life sciences / chemistry       0.00      0.00      0.00        39
                        skill_k50_business analysis / change management / integration       0.00      0.00      0.00        97
             skill_k50_coaching / leadership development / organizational development       0.00      0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [179]:
print(classification_report(y_test, (np.array(multi_target_LGBM.predict_proba(X_test))[:,:,1].T>0.01).astype(int),target_names=y_col))

                                                                                       precision    recall  f1-score   support

                                skill_k50_accounting / financial reporting / auditing       0.13      0.98      0.22        81
                                    skill_k50_analysis / financial analysis / finance       0.90      1.00      0.95       613
                              skill_k50_autocad / solidworks / mechanical engineering       0.00      0.00      0.00         8
                                               skill_k50_banking / insurance / credit       0.17      0.98      0.29       115
                                  skill_k50_biotechnology / life sciences / chemistry       0.10      0.44      0.16        39
                        skill_k50_business analysis / change management / integration       0.14      0.92      0.24        97
             skill_k50_coaching / leadership development / organizational development       0.40      0.06    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [180]:
print(classification_report(y_test, (np.array(multi_target_LGBM.predict_proba(X_test))[:,:,1].T>0.1).astype(int),target_names=y_col))

                                                                                       precision    recall  f1-score   support

                                skill_k50_accounting / financial reporting / auditing       0.15      0.30      0.20        81
                                    skill_k50_analysis / financial analysis / finance       0.90      1.00      0.95       613
                              skill_k50_autocad / solidworks / mechanical engineering       0.00      0.00      0.00         8
                                               skill_k50_banking / insurance / credit       0.18      0.50      0.26       115
                                  skill_k50_biotechnology / life sciences / chemistry       0.45      0.13      0.20        39
                        skill_k50_business analysis / change management / integration       0.13      0.14      0.14        97
             skill_k50_coaching / leadership development / organizational development       0.00      0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [181]:
print(classification_report(y_test, (np.array(multi_target_LGBM.predict_proba(X_test))[:,:,1].T>0.001).astype(int),target_names=y_col))

                                                                                       precision    recall  f1-score   support

                                skill_k50_accounting / financial reporting / auditing       0.12      1.00      0.21        81
                                    skill_k50_analysis / financial analysis / finance       0.90      1.00      0.95       613
                              skill_k50_autocad / solidworks / mechanical engineering       0.00      0.00      0.00         8
                                               skill_k50_banking / insurance / credit       0.17      1.00      0.29       115
                                  skill_k50_biotechnology / life sciences / chemistry       0.06      0.87      0.12        39
                        skill_k50_business analysis / change management / integration       0.14      1.00      0.25        97
             skill_k50_coaching / leadership development / organizational development       0.07      0.19    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
